
# Assignment 3 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 3, PROBLEM 1
Maximum Points = 8


Download the updated data folder from the course github website or just download directly the file [https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv) from the github website and put it inside your data folder, i.e. you want the path `data/smhi.csv`. The data was aquired from SMHI (Swedish Meteorological and Hydrological Institute) and constitutes per hour measurements of wind in the Uppsala Aut station. The data consists of windspeed and direction. Your goal is to load the data and work with it a bit. The code you produce should load the file as it is, please do not alter the file as the autograder will only have access to the original file.

The file information is in Swedish so you need to use some translation service, for instance `Google translate` or ChatGPT.

1. [2p] Load the file, for instance using the `csv` package. Put the wind-direction as a numpy array and the wind-speed as another numpy array.
2. [2p] Use the wind-direction which is an angle in degrees and convert it into a point on the unit circle. Store the `x_coordinate` as one array and the `y_coordinate` as another. From these coordinates, construct the wind-velocity vector.
3. [2p] Calculate the average wind velocity and convert it back to direction and compare it to just taking average of the wind direction as given in the data-file.
4. [2p] The wind velocity is a $2$-dimensional random variable, calculate the empirical covariance matrix which should be a numpy array of shape (2,2).

For you to wonder about, is it more likely for you to have headwind or not when going to the university in the morning.

In [15]:
import csv
import numpy as np

# Load the CSV file
wind_direction = []
wind_speed = []

with open("data/smhi.csv", mode='r') as file: 
    csv_reader = csv.reader(file, delimiter=';')

    for _ in range(12):
        next(csv_reader)


    for row in csv_reader:
        #print(row[2], row[4])
        wind_direction.append(float(row[2]))
        wind_speed.append(float(row[4]))

    
    
# Convert to NumPy arrays
wind_direction = np.array(wind_direction).flatten()
wind_speed = np.array(wind_speed).flatten()
print(wind_direction,wind_speed)
print(wind_direction.shape,wind_speed.shape)





problem1_wind_direction = wind_direction
problem1_wind_speed = wind_speed

[348.  43. 185. ... 269. 281. 252.] [0.6 0.2 0.1 ... 1.6 1.4 1.4]
(3061,) (3061,)


In [16]:

# The wind direction is given as a compass direction in degrees (0-360)
# convert it to x and y coordinates using the standard mathematical convention

wind_direction_radians = wind_direction * (np.pi / 180)

x_coordinate = np.cos(wind_direction_radians)
y_coordinate = np.sin(wind_direction_radians)

velocity_x = wind_speed * np.cos(wind_direction_radians)
velocity_y = wind_speed * np.sin(wind_direction_radians)

problem1_wind_direction_x_coordinate = x_coordinate
problem1_wind_direction_y_coordinate = y_coordinate

problem1_wind_velocity_x_coordinate = velocity_x
problem1_wind_velocity_y_coordinate = velocity_y

In [17]:

# Put the average wind velocity x and y coordinates here in these variables
average_velocity_x = np.mean(velocity_x)
average_velocity_y = np.mean(velocity_y)

problem1_average_wind_velocity_x_coordinate = average_velocity_x
problem1_average_wind_velocity_y_coordinate = average_velocity_y
average_wind_direction = np.rad2deg(np.arctan2(average_velocity_y, average_velocity_x))
if average_wind_direction < 0:
    average_wind_direction += 360
print(average_wind_direction)

# First calculate the angle of the average wind velocity vector in degrees
problem1_average_wind_velocity_angle_degrees = average_wind_direction
# Then calculate the average angle of the wind direction in degrees (using the wind direction in the data)
print(np.mean(wind_direction))
problem1_average_wind_direction_angle_degrees = np.mean(wind_direction)

# Finally, are they the same? Answer as a boolean value (True or False)
problem1_same_angle = False

200.12646997879915
192.281280627246


In [18]:

covariance_matrix = np.cov(velocity_x,velocity_y)
print(covariance_matrix)
problem1_wind_velocity_covariance_matrix = covariance_matrix

[[2.45331787 0.06366291]
 [0.06366291 1.89213988]]


---
## Assignment 3, PROBLEM 2
Maximum Points = 8


For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. Inside the `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [19]:


import pandas as pd
from sklearn.svm import SVC
df_train = pd.read_csv("data/indoor_train.csv")
df_train.head()


ModuleNotFoundError: No module named 'pandas'

In [ ]:
#features = df_train[['Position X', 'Position Y', 'Position Z']].to_numpy()
labels = df_train['Location'].to_numpy(dtype=int)
features = df_train.to_numpy(dtype=float)[:,0:3]
print(features)
print(labels)




Xtrain = features
Ytrain = labels

[[32.  15.   4.4]
 [ 8.  17.   1.5]
 [ 4.  13.   4.4]
 ...
 [ 4.   7.   1.5]
 [21.  19.   1.5]
 [25.  19.   7.6]]
[18  9 13 ...  2 17 19]


In [ ]:

SVM_model = SVC(kernel='linear')
svc_train = SVM_model.fit(Xtrain, Ytrain)

---
## Assignment 3, PROBLEM 3
Maximum Points = 8


Let us build a proportional model ($\mathbb{P}(Y=1 \mid X) = G(\beta_0+\beta \cdot X)$ where $G$ is the logistic function) for the spam vs not spam data. Here we assume that the features are presence vs not presence of a word, let $X_1,X_2,X_3$ denote the presence (1) or absence (0) of the words $("free", "prize", "win")$.

1. [2p] Load the file `data/spam.csv` and create two numpy arrays, `problem3_X` which has shape **(n_texts,3)** where each feature in `problem3_X` corresponds to $X_1,X_2,X_3$ from above, `problem3_Y` which has shape **(n_texts,)** and consists of a $1$ if the email is spam and $0$ if it is not. Split this data into a train-calibration-test sets where we have the split $40\%$, $20\%$, $40\%$, put this data in the designated variables in the code cell.

2. [2p] Follow the calculation from the lecture notes where we derive the logistic regression and implement the final loss function inside the class `ProportionalSpam`. You can use the `Test` cell to check that it gives the correct value for a test-point.

3. [2p] Train the model `problem3_ps` on the training data. The goal is to calibrate the probabilities output from the model. Start by creating a new variable `problem3_X_pred` (shape `(n_samples,1)`) which consists of the predictions of `problem3_ps` on the calibration dataset. Then train a calibration model using `sklearn.tree.DecisionTreeRegressor`, store this trained model in `problem3_calibrator`. Recall that calibration error is the following for a fixed function $f$
$$
    \sqrt{\mathbb{E}[|\mathbb{E}[Y \mid f(X)] - f(X)|^2]}.
$$

4. [2p] Use the trained model `problem3_ps` and the calibrator `problem3_calibrator` to make final predictions on the testing data, store the prediction in `problem3_final_predictions`. 

In [ ]:
from sklearn.model_selection import train_test_split
df = pd.read_csv("data/spam.csv", encoding='latin-1')
df = df.rename(columns={'v1': 'label', 'v2': 'message'})

#creating binary labels (1 for spam, 0 for ham)
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

print(df["message"])

df['X1'] = df['message'].str.contains("free").astype(int)
df['X2'] = df['message'].str.contains("prize").astype(int)
df['X3'] = df['message'].str.contains("win").astype(int)

problem3_X = df[['X1', 'X2', 'X3']].values
problem3_Y = df['label'].values

print(f"problem3_X: {problem3_X}")
print(f"problem3_Y: {problem3_Y}")

# Split data (40% train, 20% calibration, 40% test)
X_temp, X_test, y_temp, y_test = train_test_split(problem3_X, problem3_Y, test_size=0.4, random_state=42)
X_train, X_calibration, y_train, y_calibration = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

problem3_X_train = X_train
problem3_X_calib = X_calibration
problem3_X_test = X_test

problem3_Y_train = y_train
problem3_Y_calib = y_calibration
problem3_Y_test = y_test

print(problem3_X_train.shape,problem3_X_calib.shape,problem3_X_test.shape,problem3_Y_train.shape,problem3_Y_calib.shape,problem3_Y_test.shape)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:

from scipy.special import expit  # Logistic (sigmoid) function
from scipy import optimize

class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        # Add intercept to X
        X_with_intercept = np.c_[np.ones(X.shape[0]), X]
        # Transform Y to be +1 and -1
        Y_transformed = 2 * Y - 1
        # Linear combination z = X_with_intercept * coeffs
        z = np.dot(X_with_intercept, coeffs)
        # Logistic loss formula
        logistic_loss = np.log(1 + np.exp(-Y_transformed * z))
        # Return average loss
        return np.mean(logistic_loss)

        

    def fit(self,X,Y):
        #Use the f above together with an optimization method from scipy
        #to find the coefficients of the model
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1)
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            G = lambda x: np.exp(x)/(1+np.exp(x))
            return np.round(10*G(np.dot(X,self.coeffs[1:])+self.coeffs[0]))/10 # This rounding is to help you with the calibration

In [ ]:

# Training the logistic regression model using training data
problem3_ps = ProportionalSpam()
problem3_ps.fit(X_train, y_train)

## Step 2: Predict probabilities on the calibration set using trained model
problem3_X_pred = problem3_ps.predict(X_calibration).reshape(-1, 1)

# Step 3: Train the calibration model that maps from probabilities on the calibration set to calibration labels
from sklearn.tree import DecisionTreeRegressor
problem3_calibrator = DecisionTreeRegressor(max_depth=3, random_state=42)
problem3_calibrator.fit(problem3_X_pred, y_calibration)

DecisionTreeRegressor(max_depth=3, random_state=42)

In [ ]:

# Predict probabilities on the test set using the logistic regression model
test_probs = problem3_ps.predict(X_test).reshape(-1,1)
print(np.unique(test_probs))

# Calibrate the predicted probabilities using the calibration model
problem3_final_predictions = problem3_calibrator.predict(test_probs)
print(np.unique(problem3_final_predictions))


[0.1 0.5 0.6 0.9 1. ]
[0.09763033 0.5        0.65       1.        ]


---
#### Local Test for Assignment 3, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [ ]:
try:
    import numpy as np
    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1,0,1],[0,1,1]]),np.array([1,0]),np.array([1.2,0.4,0.3,0.9]))
    print(test_loss)
    assert (np.abs(test_loss-1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")

1.2828629432232497
Your loss was correct for a test point
